Covariant shift detection for tabular data 

This example of feature drift detection for tabular data is inspired by
https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_online_wine.html


In [4]:
import sys 
sys.path.append("../../") # Adding path to project root for this specific notebook

In [5]:
import pandas as pd
import numpy as np

red = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-red.csv", sep=';'
)
white = pd.read_csv(
    "https://storage.googleapis.com/seldon-datasets/wine_quality/winequality-white.csv", sep=';'
)
white = white.drop(["quality"], axis=1)
red = red.drop(["quality"], axis=1)
white = white[:500].astype(np.float64)
red = red[:500].astype(np.float64)



In [10]:

from drifting import DriftingClient, DriftType

client = DriftingClient()
detector_name = "WineDriftDetector"

client.fit(white, drift_type=DriftType.TABULAR, detector_name=detector_name)

<Response [500]>

In [9]:
client.load(detector_name=detector_name)

HTTPError: 404 Client Error: Not Found for url: http://localhost:8080/v2/repository/models/WineDriftDetector/load

In [27]:

drifts = []
for i in range(len(red)):
    is_drift, stat_val = client.predict(red.iloc[[i], :], drift_type=DriftType.TABULAR, detector_name=detector_name)
    # print(response)
    if is_drift:
        # The drift detector indicates after each sample if there is a drift in the data
        print(f'Change detected at index {i}')
        drifts.append(i)
    break


In [ ]:
def time_run(cd, X):
    n = X.shape[0]
    perm = np.random.permutation(n)
    t = 0
    cd.reset_state()
    while True:
        pred = cd.predict(X[perm[t%n]])
        if pred['data']['is_drift'] == 1:
            return t
        else:
            t += 1

In [ ]:
ert = 50
n_runs = 250
times_h0 = [time_run(cd, X_h0) for _ in range(n_runs)]
print(f"Average run-time under no-drift: {np.mean(times_h0)}")
_ = scipy.stats.probplot(np.array(times_h0), dist=scipy.stats.geom, sparams=1/ert, plot=plt)

In [ ]:
ert = 50
n_runs = 250
times_h1 = [time_run(cd, X_corr) for _ in range(n_runs)]
print(f"Average run-time under drift: {np.mean(times_h1)}")